# Firefox Launcher Extension Test Notebook

This notebook tests and validates the installation of the Firefox launcher extension in your JupyterHub environment.

## Overview
The Firefox launcher extension provides a way to run Firefox through Xpra HTML5 sessions directly from JupyterLab. This notebook will verify that all components are properly installed and configured.

## Test Categories
1. **Extension Installation Status** - Check if extensions are installed and enabled
2. **Server Extension Configuration** - Validate server-side components
3. **Lab Extension Components** - Verify frontend components
4. **Server Proxy Setup** - Test jupyter-server-proxy integration
5. **System Dependencies** - Check Firefox and Xpra availability
6. **Entry Points** - Verify extension registration
7. **Process Management** - Test socket and port capabilities

## 1. Import Required Libraries

Import all necessary modules for testing the extension installation.

In [ ]:
import subprocess
import sys
import os
import json
from shutil import which
from pathlib import Path

print("✅ Successfully imported required libraries")
print(f"📍 Current working directory: {os.getcwd()}")
print(f"🐍 Python version: {sys.version}")
print(f"📁 Home directory: {os.path.expanduser('~')}")

## 2. Check Extension Installation Status

Verify that both the server extension and lab extension are properly installed and enabled.

In [ ]:
# Check server extension
print("🔍 Checking server extensions...")
result = subprocess.run(['jupyter', 'server', 'extension', 'list'], capture_output=True, text=True)

if 'jupyterlab_firefox_launcher' in result.stdout:
    print("✅ Server extension: jupyterlab_firefox_launcher is installed")
    if 'enabled' in result.stdout and 'jupyterlab_firefox_launcher' in result.stdout:
        print("✅ Server extension: ENABLED")
    else:
        print("⚠️  Server extension: installed but may not be enabled")
else:
    print("❌ Server extension: NOT FOUND")

print("\n📋 All server extensions:")
for line in result.stdout.split('\n'):
    if line.strip() and ('enabled' in line.lower() or 'disabled' in line.lower()):
        print(f"   {line.strip()}")

In [ ]:
# Check lab extension
print("🔍 Checking lab extensions...")
result = subprocess.run(['jupyter', 'labextension', 'list'], capture_output=True, text=True)

if 'jupyterlab-firefox-launcher' in result.stdout:
    print("✅ Lab extension: jupyterlab-firefox-launcher is installed")
    for line in result.stdout.split('\n'):
        if 'jupyterlab-firefox-launcher' in line:
            print(f"   📊 Status: {line.strip()}")
            break
else:
    print("❌ Lab extension: NOT FOUND")

print("\n📋 All lab extensions:")
for line in result.stdout.split('\n'):
    if line.strip() and ('enabled' in line.lower() or 'disabled' in line.lower() or 'installed' in line.lower()):
        print(f"   {line.strip()}")

## 3. Test Server Extension Configuration

Import and test the server extension module to ensure it loads correctly.

In [ ]:
try:
    # Try to import the server extension
    from jupyterlab_firefox_launcher import __init__ as server_init
    print("✅ Server extension module imports successfully")
    
    # Try to import the server proxy configuration
    from jupyterlab_firefox_launcher.server_proxy import setup_firefox_desktop
    print("✅ Server proxy module imports successfully")
    
    # Try to import the new server extension
    from jupyterlab_firefox_launcher.new_server_extension import _load_jupyter_server_extension
    print("✅ New server extension module imports successfully")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("   This indicates the server extension is not properly installed")
except Exception as e:
    print(f"⚠️  Unexpected error: {e}")

## 4. Verify Lab Extension Components

Check that the lab extension files are properly built and accessible.

In [ ]:
# Check for lab extension files
extension_files = [
    'package.json',
    'frontend/src/index.ts',
    'lib/index.js',  # Built JavaScript files
    'dist/',         # Distribution directory
]

print("🔍 Checking lab extension files...")
for file_path in extension_files:
    full_path = Path(file_path)
    if full_path.exists():
        if full_path.is_file():
            print(f"✅ Found file: {file_path}")
        else:
            print(f"✅ Found directory: {file_path}")
    else:
        print(f"❌ Missing: {file_path}")

# Check package.json content
try:
    with open('package.json', 'r') as f:
        package_data = json.load(f)
    
    print(f"\n📦 Package name: {package_data.get('name', 'unknown')}")
    print(f"🔢 Version: {package_data.get('version', 'unknown')}")
    print(f"📝 Description: {package_data.get('description', 'none')}")
    
    # Check for JupyterLab extension metadata
    if 'jupyterlab' in package_data:
        print("✅ JupyterLab extension metadata found")
        jlab_config = package_data['jupyterlab']
        if 'extension' in jlab_config:
            print(f"   🎯 Extension entry point: {jlab_config['extension']}")
    else:
        print("⚠️  No JupyterLab extension metadata found")
        
except FileNotFoundError:
    print("❌ package.json not found")
except json.JSONDecodeError:
    print("❌ package.json is not valid JSON")

## 5. Test Server Proxy Setup

Import and execute the setup_firefox_desktop function to validate the server proxy configuration.

In [ ]:
try:
    from jupyterlab_firefox_launcher.server_proxy import setup_firefox_desktop
    
    # Call the setup function
    config = setup_firefox_desktop()
    print("✅ Server proxy configuration loads successfully")
    
    # Display configuration details
    print(f"\n🏷️  Launcher entry title: {config['launcher_entry']['title']}")
    print(f"🔧 Command (first 3 args): {' '.join(config['command'][:3])}")
    print(f"🌐 Port configuration: {config.get('port', 'dynamic')}")
    print(f"📁 Working directory: {config.get('new_browser_window_path', 'default')}")
    print(f"⏱️  Timeout: {config.get('timeout', 'default')} seconds")
    
    # Check specific Xpra configuration
    command_str = ' '.join(config['command'])
    print(f"\n🔍 Command analysis:")
    
    if 'xpra' in command_str:
        print("   ✅ Uses Xpra")
    if '--socket-dirs=' in command_str:
        print("   ✅ User-space socket configuration")
    if '--system-proxy-socket=no' in command_str:
        print("   ✅ System proxy socket disabled")
    if '--daemon=no' in command_str:
        print("   ✅ Daemon mode disabled")
    if '--start=' in command_str:
        print("   ✅ Firefox wrapper script configured")
    
    print(f"\n📋 Full command preview:")
    for i, arg in enumerate(config['command'][:10]):  # Show first 10 arguments
        print(f"   [{i}] {arg}")
    if len(config['command']) > 10:
        print(f"   ... and {len(config['command']) - 10} more arguments")
        
except Exception as e:
    print(f"❌ Server proxy configuration failed: {e}")
    print(f"   Error type: {type(e).__name__}")
    import traceback
    print(f"   Traceback: {traceback.format_exc()}")

## 6. Validate Firefox and Xpra Dependencies

Check that Firefox and Xpra are available and can be executed.

In [ ]:
# Check system dependencies
dependencies = {
    'xpra': 'Xpra (remote display server)',
    'firefox': 'Firefox browser',
    'jupyter': 'Jupyter'
}

print("🔍 Checking system dependencies...")
for cmd, desc in dependencies.items():
    cmd_path = which(cmd)
    if cmd_path:
        print(f"✅ {desc}: {cmd_path}")
        
        # Try to get version information
        try:
            if cmd == 'xpra':
                result = subprocess.run([cmd, '--version'], capture_output=True, text=True, timeout=5)
                if result.returncode == 0:
                    version = result.stdout.split('\n')[0]
                    print(f"   📊 Version: {version}")
            elif cmd == 'firefox':
                result = subprocess.run([cmd, '--version'], capture_output=True, text=True, timeout=5)
                if result.returncode == 0:
                    print(f"   📊 Version: {result.stdout.strip()}")
            elif cmd == 'jupyter':
                result = subprocess.run([cmd, '--version'], capture_output=True, text=True, timeout=5)
                if result.returncode == 0:
                    print(f"   📊 Version: {result.stdout.strip()}")
        except Exception as e:
            print(f"   ⚠️  Could not get version: {e}")
    else:
        print(f"❌ {desc}: NOT FOUND in PATH")

# Check for Xpra HTML5 support
print(f"\n🔍 Checking for Xpra HTML5 support...")
html5_paths = [
    '/usr/share/xpra/www',
    '/usr/local/share/xpra/www',
    '/opt/xpra/share/xpra/www'
]

html5_found = False
for path in html5_paths:
    if os.path.exists(path):
        print(f"✅ Xpra HTML5 support found: {path}")
        html5_found = True
        break

if not html5_found:
    print("⚠️  Xpra HTML5 support not found in standard locations")
    print("   Extension may still work if HTML5 support is installed elsewhere")

## 7. Test Extension Entry Points

Verify that the extension entry points are properly registered with jupyter-server-proxy.

In [ ]:
# Check if jupyter-server-proxy is available
try:
    import jupyter_server_proxy
    print(f"✅ jupyter-server-proxy is installed: {jupyter_server_proxy.__version__}")
except ImportError:
    print("❌ jupyter-server-proxy is NOT installed")
    print("   Install with: pip install jupyter-server-proxy")

# Check entry points using pkg_resources
try:
    import pkg_resources
    
    print(f"\n🔍 Checking jupyter-server-proxy entry points...")
    
    # Look for our entry point
    entry_points = pkg_resources.get_entry_map('jupyterlab-firefox-launcher', 'jupyter_serverproxy_servers')
    
    if entry_points:
        print("✅ Found jupyter-server-proxy entry points:")
        for name, entry_point in entry_points.items():
            print(f"   📌 {name}: {entry_point}")
    else:
        print("❌ No jupyter-server-proxy entry points found")
        
        # Check if the package is installed at all
        try:
            dist = pkg_resources.get_distribution('jupyterlab-firefox-launcher')
            print(f"   📦 Package is installed: {dist.version}")
            print("   ⚠️  But no entry points found - check pyproject.toml")
        except pkg_resources.DistributionNotFound:
            print("   📦 Package not found - extension may not be properly installed")
            
except ImportError:
    print("⚠️  pkg_resources not available, using alternative method")
    
    # Alternative: check pyproject.toml directly
    try:
        import tomllib if sys.version_info >= (3, 11) else None
        if tomllib is None:
            import tomli as tomllib
            
        with open('pyproject.toml', 'rb') as f:
            pyproject_data = tomllib.load(f)
            
        entry_points = pyproject_data.get('project', {}).get('entry-points', {})
        server_proxy_entries = entry_points.get('jupyter_serverproxy_servers', {})
        
        if server_proxy_entries:
            print("✅ Found entry points in pyproject.toml:")
            for name, entry_point in server_proxy_entries.items():
                print(f"   📌 {name}: {entry_point}")
        else:
            print("❌ No jupyter-server-proxy entry points in pyproject.toml")
            
    except Exception as e:
        print(f"⚠️  Could not check pyproject.toml: {e}")

## 8. Check Process and Port Management

Test socket creation, port binding, and process management capabilities.

In [ ]:
import socket
import tempfile

print("🔍 Testing process and port management capabilities...")

# Test socket directory creation
socket_dirs = [
    os.path.expanduser("~/.xpra-sockets"),
    os.path.expanduser("~/.firefox-launcher-profiles")
]

for socket_dir in socket_dirs:
    try:
        os.makedirs(socket_dir, exist_ok=True)
        print(f"✅ Can create directory: {socket_dir}")
        
        # Test write permissions
        test_file = os.path.join(socket_dir, "test_write")
        with open(test_file, 'w') as f:
            f.write("test")
        os.remove(test_file)
        print(f"   ✅ Write permissions OK")
        
    except Exception as e:
        print(f"❌ Cannot create/write to {socket_dir}: {e}")

# Test port binding
print(f"\n🔌 Testing port binding...")
try:
    # Test binding to a random port
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('127.0.0.1', 0))  # Bind to any available port
    port = sock.getsockname()[1]
    sock.close()
    print(f"✅ Can bind to local ports (tested port {port})")
    
    # Test binding to multiple ports (simulate jupyter-server-proxy behavior)
    test_ports = []
    for i in range(3):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1', 0))
        port = sock.getsockname()[1]
        test_ports.append((sock, port))
    
    print(f"✅ Can bind to multiple ports: {[p[1] for p in test_ports]}")
    
    # Clean up
    for sock, port in test_ports:
        sock.close()
        
except Exception as e:
    print(f"❌ Port binding test failed: {e}")

# Test environment variables that affect process management
print(f"\n🌍 Environment variables:")
env_vars = ['DISPLAY', 'PATH', 'HOME', 'USER', 'JUPYTER_SERVER_ROOT']
for var in env_vars:
    value = os.environ.get(var)
    if value:
        if var == 'PATH':
            print(f"✅ {var}: ({len(value.split(':'))} paths)")
        else:
            print(f"✅ {var}: {value}")
    else:
        print(f"ℹ️  {var}: Not set")

print(f"\n🏠 User directories:")
print(f"   Home: {os.path.expanduser('~')}")
print(f"   Current: {os.getcwd()}")
print(f"   Temp: {tempfile.gettempdir()}")

## Summary and Next Steps

Run all the cells above to validate your Firefox launcher extension installation. 

### If all tests pass ✅
- Look for "Firefox Browser" in the JupyterLab Launcher
- Click it to start a Firefox session
- The session will open in a new browser tab via Xpra HTML5

### If tests fail ❌
Common fixes:
- **Missing extensions**: Run `pip install -e .` and `jupyter labextension develop . --overwrite`
- **Missing dependencies**: Install with `apt install xpra xpra-html5 firefox` (Ubuntu)
- **jupyter-server-proxy**: Install with `pip install jupyter-server-proxy`
- **Enable extensions**: Run `jupyter server extension enable jupyterlab_firefox_launcher`

### Troubleshooting Tips
- Check JupyterLab browser console for JavaScript errors
- Look at JupyterLab server logs for Python errors
- Verify that the "Firefox Browser" launcher appears in the JupyterLab Launcher tab
- Test Xpra manually: `xpra start --html=on --bind-tcp=0.0.0.0:8080`

## 🔧 NFS Permission Fix Test

Re-test the server proxy configuration after fixing the NFS permission issue.

In [ ]:
# Reload the module to get the updated code
import importlib
import jupyterlab_firefox_launcher.server_proxy
importlib.reload(jupyterlab_firefox_launcher.server_proxy)

try:
    from jupyterlab_firefox_launcher.server_proxy import setup_firefox_desktop
    
    # Call the setup function with the NFS fix
    config = setup_firefox_desktop()
    print("✅ Server proxy configuration now works!")
    
    # Display configuration details
    print(f"\n🏷️  Launcher entry title: {config['launcher_entry']['title']}")
    print(f"🔧 Command (first 3 args): {' '.join(config['command'][:3])}")
    print(f"🌐 Port configuration: {config.get('port', 'dynamic')}")
    
    # Check specific configuration
    command_str = ' '.join(config['command'])
    print(f"\n🔍 Configuration analysis:")
    
    if 'xpra' in command_str:
        print("   ✅ Uses Xpra")
    if '--start=' in command_str:
        wrapper_path = None
        for i, arg in enumerate(config['command']):
            if '--start=' in arg:
                wrapper_path = arg.split('=', 1)[1]
                break
        if wrapper_path:
            print(f"   ✅ Firefox wrapper: {wrapper_path}")
            if wrapper_path.startswith('/nfs/'):
                print("   ⚠️  Still using NFS path - may have permission issues")
            elif wrapper_path.startswith(os.path.expanduser('~')):
                print("   ✅ Using user-space wrapper (NFS issue avoided)")
    
    print(f"\n📋 Full command preview:")
    for i, arg in enumerate(config['command'][:8]):  # Show first 8 arguments
        print(f"   [{i}] {arg}")
    if len(config['command']) > 8:
        print(f"   ... and {len(config['command']) - 8} more arguments")
        
except Exception as e:
    print(f"❌ Server proxy configuration still failed: {e}")
    print(f"   Error type: {type(e).__name__}")
    import traceback
    print(f"   Traceback: {traceback.format_exc()}")

## 🐛 Debug Firefox Startup Command

Test the actual command that gets executed to identify startup issues.

In [ ]:
import subprocess
import tempfile
import time

# Get the configuration
try:
    from jupyterlab_firefox_launcher.server_proxy import setup_firefox_desktop
    config = setup_firefox_desktop()
    
    print("🔍 Debugging Firefox startup command...")
    print(f"Command: {config['command']}")
    print(f"Timeout: {config['timeout']} seconds")
    
    # Parse the actual shell command
    if config['command'][0] == 'sh' and config['command'][1] == '-c':
        shell_command = config['command'][2]
        print(f"\n📋 Shell command to execute:")
        print(shell_command)
        
        # Extract the xpra command (after 'cd ... && ')
        if ' && ' in shell_command:
            xpra_command = shell_command.split(' && ', 1)[1]
            print(f"\n🔧 Xpra command:")
            print(xpra_command)
            
            # Replace {port} with a test port for debugging
            test_port = 19876
            debug_command = xpra_command.replace('{port}', str(test_port))
            print(f"\n🧪 Debug command (port {test_port}):")
            print(debug_command)
            
            # Test just the firefox wrapper first
            wrapper_test = None
            if '--start=' in debug_command:
                for part in debug_command.split():
                    if '--start=' in part:
                        wrapper_path = part.split('=', 1)[1]
                        print(f"\n🎯 Testing Firefox wrapper: {wrapper_path}")
                        
                        # Check if wrapper exists and is executable
                        import os
                        if os.path.exists(wrapper_path):
                            stat_info = os.stat(wrapper_path)
                            is_executable = stat_info.st_mode & 0o111
                            print(f"   ✅ Wrapper exists: {wrapper_path}")
                            print(f"   {'✅' if is_executable else '❌'} Executable: {bool(is_executable)}")
                            
                            # Test wrapper execution
                            try:
                                result = subprocess.run([wrapper_path, '--version'], 
                                                      capture_output=True, text=True, timeout=10)
                                if result.returncode == 0:
                                    print(f"   ✅ Wrapper test successful")
                                else:
                                    print(f"   ⚠️  Wrapper test failed: {result.stderr}")
                            except Exception as e:
                                print(f"   ❌ Wrapper test error: {e}")
                        else:
                            print(f"   ❌ Wrapper not found: {wrapper_path}")
                        break
                        
            # Test xpra availability
            print(f"\n🔧 Testing Xpra availability:")
            try:
                result = subprocess.run(['xpra', '--version'], capture_output=True, text=True, timeout=5)
                if result.returncode == 0:
                    print(f"   ✅ Xpra available: {result.stdout.strip()}")
                else:
                    print(f"   ❌ Xpra test failed")
            except Exception as e:
                print(f"   ❌ Xpra not available: {e}")
                
            # Test a minimal xpra command (without firefox)
            print(f"\n🧪 Testing minimal Xpra start...")
            minimal_cmd = f"xpra start --bind-tcp=127.0.0.1:{test_port} --html=on --daemon=no --exit-with-children=yes"
            print(f"Command: {minimal_cmd}")
            
            try:
                # Start the process but don't wait - just check if it starts
                proc = subprocess.Popen(minimal_cmd.split(), 
                                      stdout=subprocess.PIPE, 
                                      stderr=subprocess.PIPE)
                time.sleep(3)  # Give it a moment to start
                
                if proc.poll() is None:
                    print(f"   ✅ Xpra started successfully (PID: {proc.pid})")
                    proc.terminate()  # Clean up
                    proc.wait()
                else:
                    stdout, stderr = proc.communicate()
                    print(f"   ❌ Xpra failed to start")
                    print(f"   Stdout: {stdout.decode()}")
                    print(f"   Stderr: {stderr.decode()}")
                    
            except Exception as e:
                print(f"   ❌ Xpra start test failed: {e}")
                
except Exception as e:
    print(f"❌ Debug failed: {e}")
    import traceback
    print(traceback.format_exc())